In [1]:
# Set Up Drive, Kaggle  and Download Dataset
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# verify kaggle works
!kaggle datasets list

# Download the dataset (12K images)
!kaggle datasets download -d abdelrahmanemad2199/corn-or-maize-leaf-disease-dataset

# Unzip the dataset (replace 'dataset-identifier.zip' with the actual downloaded zip file name
!unzip corn-or-maize-leaf-disease-dataset.zip -d /content/dataset

from google.colab import drive
drive.mount('/content/drive')

# Move the dataset to your Google Drive
!mv /content/dataset /content/drive/MyDrive/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8da48548-461c-4574-9435-74310d8207a3___RS_GLSp 4387_90deg.JPG  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8da48548-461c-4574-9435-74310d8207a3___RS_GLSp 4387_new30degFlipLR.JPG  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8ead1e08-eb49-4c03-b99f-f46f3597087d___RS_GLSp 4474 copy.jpg  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8ead1e08-eb49-4c03-b99f-f46f3597087d___RS_GLSp 4474 copy_270deg.jpg  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8ead1e08-eb49-4c03-b99f-f46f3597087d___RS_GLSp 4474 copy_90deg.jpg  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/8ead1e08-eb49-4c03-b99f-f46f3597087d___RS_GLSp 4474 copy_new30degFlipLR.jpg  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/9043a294-8d22-405f-add1-7e5bf1606e7a___RS_GLSp 4431 copy 2.jpg  
  inflating: /content/dataset/corn/data/Gray_Leaf_Spot/9043a294-8d22-405f-add1-7e5

In [3]:
# Data Preprocessing
import tensorflow as tf

dataset_dir = '/content/drive/MyDrive/dataset/corn/data'

# Define the image size
image_size = (256, 256)

# Define batch size
batch_size = 32

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# Load the dataset for training
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

# Load the dataset for validation
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)


Found 9889 images belonging to 4 classes.
Found 2471 images belonging to 4 classes.


In [4]:
# Model Compilation with Transfer Learning

import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from keras.optimizers import Adam


base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Assuming 4 classes for maize leaf diseases
])



initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)
optimizer = tf.keras.optimizers.Nadam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])





9406464/9406464 [==============================] - 2s 0us/step


In [ ]:
# Training

import os

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/a1.keras', save_best_only=True, )

# Check if the checkpoint file exists
if os.path.exists('/content/drive/MyDrive/a1.keras'):
    # Load the best weights saved by the checkpoint
    model.load_weights('/content/drive/MyDrive/a1.keras')
    print("Best model weights loaded successfully!")
else:
    print("No previous checkpoint found. Training from scratch...")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

No previous checkpoint found. Training from scratch...
Epoch 1/50
104/104 [==============================] - 114s 1s/step - loss: 0.4892 - accuracy: 0.8051 - val_loss: 0.2350 - val_accuracy: 0.9050
Epoch 2/50
104/104 [==============================] - 125s 1s/step - loss: 0.2937 - accuracy: 0.8843 - val_loss: 0.2416 - val_accuracy: 0.9038
Epoch 3/50
104/104 [==============================] - 99s 951ms/step - loss: 0.2581 - accuracy: 0.8898 - val_loss: 0.2253 - val_accuracy: 0.9123
Epoch 4/50
104/104 [==============================] - 101s 975ms/step - loss: 0.2473 - accuracy: 0.9036 - val_loss: 0.2111 - val_accuracy: 0.9207
Epoch 5/50
104/104 [==============================] - 121s 1s/step - loss: 0.2295 - accuracy: 0.9081 - val_loss: 0.2425 - val_accuracy: 0.8930
Epoch 6/50
104/104 [==============================] - 101s 974ms/step - loss: 0.2312 - accuracy: 0.9081 - val_loss: 0.2303 - val_accuracy: 0.8906
Epoch 7/50
104/104 [==============================] - 121s 1s/step - loss: 0.22

In [ ]:
# Evaluate model

test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

27/27 [==============================] - 21s 783ms/step - loss: 0.1785 - accuracy: 0.9282
Test accuracy: 0.9282296895980835


In [ ]:
# save the model

model.save('/content/drive/MyDrive/final_model')


In [ ]:
predictions = model.predict(validation_generator)

27/27 [==============================] - 21s 717ms/step


In [ ]:
predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)

NameError: name 'np' is not defined

In [ ]:
print(validation_generator.class_indices)

{'Blight': 0, 'Common_Rust': 1, 'Gray_Leaf_Spot': 2, 'Healthy': 3}


In [ ]:
model.save('/content/drive/MyDrive/maizespy_model.keras')